In [72]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import folium
from geopy.geocoders import Nominatim
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
import geopandas as pd   
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import geopandas as gpd
import os

### Define Foursquare Credentials and Version

In [54]:
CLIENT_ID = '5ESZXISFGD2ZO5NQC0DUKHXTYIT4PQT0U01WQYLD1PLEITLW' # your Foursquare ID
CLIENT_SECRET = 'EJFLXNIT4T52K22ZWDJBV0CSMMZCFC2VHKC2XWXAWZXUJAKN' # your Foursquare Secret
VERSION = '20200210'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ESZXISFGD2ZO5NQC0DUKHXTYIT4PQT0U01WQYLD1PLEITLW
CLIENT_SECRET:EJFLXNIT4T52K22ZWDJBV0CSMMZCFC2VHKC2XWXAWZXUJAKN


In [103]:
address = 'Madison Park, SEATTLE, WA'

geolocator = Nominatim(user_agent="foursquare_agent",timeout=3)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.6359301 -122.2801963


In [104]:
search_query = ''
LIMIT = 100
radius = 1500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

In [105]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",1544491,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/madison-kit...,False,50fedb1be4b0b707e97558a5,4122 E Madison St,...,"[4122 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63534667506617...",47.635347,-122.278819,NaN,98112,WA,Madison Kitchen,v-1581918481,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bddd5b80ee3a593c5d62eb0,E Madison St. & E Howe St.,...,"[E Madison St. & E Howe St., Seattle, WA 98112...","[{'label': 'display', 'lat': 47.63546187577772...",47.635462,-122.277306,NaN,98112,WA,Madison Park,v-1581918481,NaN
2,"[{'id': '4bf58dd8d48988d178941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,False,4d1a1589bb64224b2489d365,NaN,...,"[Washington, United States]","[{'label': 'display', 'lat': 47.63589458930579...",47.635895,-122.278686,NaN,NaN,Washington,Woodside & Woodside,v-1581918481,NaN
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",1566444,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/red-onion-t...,False,40b13b00f964a520edf61ee3,4210 E Madison St,...,"[4210 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63591922933824...",47.635919,-122.278126,NaN,98112,WA,Red Onion Tavern,v-1581918481,NaN
4,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae21029f964a520598a21e3,1837 42nd Ave E,...,"[1837 42nd Ave E (E Madison St), Seattle, WA 9...","[{'label': 'display', 'lat': 47.635711, 'lng':...",47.635711,-122.278800,NaN,98112,WA,Madison Park Hardware,v-1581918481,NaN


In [107]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Madison Kitchen,Café,4122 E Madison St,US,Seattle,United States,NaN,122,"[4122 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63534667506617...",47.635347,-122.278819,NaN,98112,WA,50fedb1be4b0b707e97558a5
1,Madison Park,Park,E Madison St. & E Howe St.,US,Seattle,United States,NaN,222,"[E Madison St. & E Howe St., Seattle, WA 98112...","[{'label': 'display', 'lat': 47.63546187577772...",47.635462,-122.277306,NaN,98112,WA,4bddd5b80ee3a593c5d62eb0
2,Woodside & Woodside,Dentist's Office,NaN,US,NaN,United States,NaN,113,"[Washington, United States]","[{'label': 'display', 'lat': 47.63589458930579...",47.635895,-122.278686,NaN,NaN,Washington,4d1a1589bb64224b2489d365
3,Red Onion Tavern,Bar,4210 E Madison St,US,Seattle,United States,NaN,155,"[4210 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63591922933824...",47.635919,-122.278126,NaN,98112,WA,40b13b00f964a520edf61ee3
4,Madison Park Hardware,Hardware Store,1837 42nd Ave E,US,Seattle,United States,E Madison St,107,"[1837 42nd Ave E (E Madison St), Seattle, WA 9...","[{'label': 'display', 'lat': 47.635711, 'lng':...",47.635711,-122.278800,NaN,98112,WA,4ae21029f964a520598a21e3
5,Junior League of Seattle,Office,4119 E Madison St,US,Seattle,United States,NaN,145,"[4119 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63503574922383...",47.635036,-122.278782,NaN,98112,WA,4b469c57f964a520772526e3
6,Bert's Red Apple,Food & Drink Shop,1801 41st Ave E,US,Seattle,United States,NaN,132,"[1801 41st Ave E, Seattle, WA 98112, United St...","[{'label': 'display', 'lat': 47.63476908497122...",47.634769,-122.279833,NaN,98112,WA,4a3844ddf964a520dc9e1fe3
7,Cookin' at Madison Park,Kitchen Supply Store,4224 E Madison St,US,Seattle,United States,NaN,179,"[4224 E Madison St, Seattle, WA 98112, United ...","[{'label': 'display', 'lat': 47.63624, 'lng': ...",47.636240,-122.277843,NaN,98112,WA,4bc8a51c14d79521338369e9
8,Starbucks,Coffee Shop,4000 East Madison St,US,Seattle,United States,NaN,208,"[4000 East Madison St, Seattle, WA 98112, Unit...","[{'label': 'display', 'lat': 47.634098, 'lng':...",47.634098,-122.280780,NaN,98112,WA,44e6e49af964a52081371fe3
9,Madison Books,Bookstore,NaN,US,Seattle,United States,NaN,103,"[Seattle, WA 98112, United States]","[{'label': 'display', 'lat': 47.635346, 'lng':...",47.635346,-122.279115,NaN,98112,WA,5cd883b29cadd90039b0f7d5


In [1]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel
# add a red circle marker to represent the Central
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Madison Park',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

NameError: name 'folium' is not defined

In [108]:
dataframe_filtered.shape

(100, 16)

In [109]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
       radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        
    ).add_to(venues_map)

# display map
venues_map

In [110]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=5ESZXISFGD2ZO5NQC0DUKHXTYIT4PQT0U01WQYLD1PLEITLW&client_secret=EJFLXNIT4T52K22ZWDJBV0CSMMZCFC2VHKC2XWXAWZXUJAKN&v=20200210&ll=47.6359301,-122.2801963&radius=1500&limit=100


In [111]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4a2911edbcad001beccca3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Madison Park',
  'headerFullLocation': 'Madison Park, Seattle',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 56,
  'suggestedBounds': {'ne': {'lat': 47.64943011350002,
    'lng': -122.26019923242839},
   'sw': {'lat': 47.62243008649999, 'lng': -122.30019336757161}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '44e6e49af964a52081371fe3',
       'name': 'Starbucks',
       'location': {'address': '4000 East Madison St',
        'lat': 47.634098,
        'lng': -122.28078,
        'labeledLatLngs': [{'label': 'd

In [112]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [113]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,47.634098,-122.280780
1,Madison Park Beach,Beach,47.635854,-122.276765
2,Madison Kitchen,Café,47.635347,-122.278819
3,Bert's Red Apple,Food & Drink Shop,47.634769,-122.279833
4,Cactus Madison Park,Mexican Restaurant,47.636196,-122.277686


In [114]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

56 venues were returned by Foursquare.


## Explore Neighborhoods in Madison Park

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [66]:

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


NameError: name 'manhattan_data' is not defined

In [92]:
nearby_venues

,name,categories,lat,lng
0,Starbucks,Coffee Shop,47.634098,-122.280780
1,Madison Park Beach,Beach,47.635854,-122.276765
2,Madison Kitchen,Café,47.635347,-122.278819
3,Bert's Red Apple,Food & Drink Shop,47.634769,-122.279833
4,Cactus Madison Park,Mexican Restaurant,47.636196,-122.277686
5,Madison Park,Park,47.635462,-122.277306
6,Scoop Du Jour,Ice Cream Shop,47.634112,-122.280424
7,Madison Park Bakery,Bakery,47.636007,-122.277886
8,The Attic Alehouse & Eatery,Bar,47.636297,-122.277584
9,The Independent Pizzeria,Pizza Place,47.636125,-122.277385


In [115]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,1,1,1
Bakery,2,2,2
Bank,2,2,2
Bar,3,3,3
Bath House,1,1,1
Beach,1,1,1
Bike Trail,1,1,1
Boat or Ferry,1,1,1
Botanical Garden,1,1,1


In [94]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel
folium.GeoJson(neigh_geo).add_to(venues_map)
# add the Italian restaurants as blue circle markers
for lat, lng, label,name in zip(nearby_venues.lat, nearby_venues.lng,nearby_venues.categories,nearby_venues.name):
    folium.CircleMarker(
        [lat, lng],
       radius=5,
        popup='cate:{} name:{}'.format(label,name),
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        
    ).add_to(venues_map)

# display map
venues_map.save('test1.html')

In [79]:
neigh_geo = os.path.join('C:/Users/12370/Desktop/data_science/neighborhoods (1).geojson')